# Reading Dutch Datasets

`pastas` comes with several helpful functions for reading data from files from common data sources in the Netherlands. These sources include:
- [Dinoloket](https://www.dinoloket.nl/ondergrondgegevens): for groundwater and surface water level timeseries.
- [KNMI](http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi): for precipitation and timeseries timeseries
- Waterbase: for surface water level timeseries from Rijkswaterstaat.
- [Menyanthes](https://www.kwrwater.nl/tools-producten/menyanthes/): read files created by Menyanthes

The functionality for reading files from Dinoloket is shown in this notebook. 

First, start with the necessesary imports.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import pastas as ps

ps.show_versions()

For reading groundwater level timeseries there is the `read_dino` function. For river level gauge data there is the `read_dino_level_gauge` function. These functions are available at the module level. The code below loads these timeseries from CSV files in the data directory.

In [ ]:
gw_levels = ps.read_dino(r'../data/B49F0555001_1.csv')
river_levels = ps.read_dino_level_gauge(r'../data/P43H0001.csv')

Note that these `pastas.Timeseries` objects contain metadata, i.e. for the `river_levels`:

In [ ]:
river_levels.metadata

The resulting `pastas.TimeSeries` come with their own plot methods, to quickly visualize the data:

In [ ]:
ax = gw_levels.plot()
river_levels.plot()

As can be observed in the plot above, something went wrong with the registration of the river levels in the field. The shows a huge downward shift at the end of 2014. The negative values from end of 2014 onwards are assummed to be correct. The positive values were registered incorrectly (missing a minus sign). 

We fix the timeseries by updating the `TimeSeries` attribute called `series_original`. The `update_series` method is called in order to renew derived attributes.

In [ ]:
river_levels.series_original = river_levels.series_original.abs() * -1  # set positive values to negative
river_levels.update_series()

Plot the timeseries again, to see if the applied fix looks reasonable:

In [ ]:
gw_levels.plot()
river_levels.plot()

Another option for loading river levels is the `DinoPeilschaal` reader class. This creates a `DinoPeilschaal` object containing the data and the metadata. These are accessible through the `data` and `meta` attributes, respectively. 

The difference with the previous method is that the original data is available as a pandas.DataFrame when using this method.


In [ ]:
river_levels_obj = ps.read.dinoloket.DinoPeilschaal(r'../data/P43H0001.csv')

View the original data:

In [ ]:
river_levels_obj.data.head()

The metadata:

In [ ]:
river_levels_obj.meta